# 🚀 High-Risk Leveraged MT5 Trading Bot (SMA Crossover Strategy)

**⚠️ DISCLAIMER: This notebook is for educational and demo purposes only. High-risk leveraged trading can result in significant losses. Never use real capital without proper backtesting and risk management.**

This comprehensive notebook demonstrates building a high-risk leveraged trading bot using the MetaTrader 5 platform in Python. The strategy uses a simple moving average (SMA) crossover technique on 5-minute candles with aggressive parameters.

## 🎯 Strategy Overview
- **Strategy**: SMA Crossover (Fast=5, Slow=20)
- **Timeframe**: 5-minute candles (M5)
- **Leverage**: 1:5
- **Lot Size**: 1.0 (high-risk)
- **Stop Loss**: 10 pips
- **Take Profit**: 20 pips
- **Frequency**: Every 60 seconds

---

## 📚 Table of Contents
1. [Installation & Setup](#Installation-&-Setup)
2. [Configuration & Parameters](#Configuration-&-Parameters)
3. [MT5 Connection](#MT5-Connection)
4. [Data Fetching & Visualization](#Data-Fetching-&-Visualization)
5. [SMA Crossover Strategy](#SMA-Crossover-Strategy)
6. [Trade Execution Engine](#Trade-Execution-Engine)
7. [Backtesting Module](#Backtesting-Module)
8. [Live Trading Bot](#Live-Trading-Bot)
9. [Trade Logging & Export](#Trade-Logging-&-Export)
10. [Parameter Tuning](#Parameter-Tuning)
11. [Risk Management](#Risk-Management)
12. [Performance Monitoring](#Performance-Monitoring)


## 🔧 Installation & Setup

First, install all required libraries. Run this command in your terminal:

```bash
pip install MetaTrader5 pandas matplotlib numpy seaborn plotly
```

If you encounter issues, try:
```bash
pip install --upgrade MetaTrader5
pip install --upgrade pandas
```

In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import time


## MT5 Initialization

Connect to your MetaTrader 5 account and verify the connection.


In [ ]:
ACCOUNT = 'YOUR_ACCOUNT'
PASSWORD = 'YOUR_PASSWORD'
SERVER = 'YOUR_SERVER'

mt5.initialize()
account_info = mt5.account_info()
if account_info is None:
    print("Failed to connect to MT5")
else:
    print("Connected to MT5 account " + str(account_info.login))
    print("Balance: " + str(account_info.balance))


## Data Fetching

Fetch historical price data required for strategy execution.


In [ ]:
symbol = 'EURUSD'
timeframe = mt5.TIMEFRAME_M5
rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, 200)
df = pd.DataFrame(rates)
df['time'] = pd.to_datetime(df['time'], unit='s')
df.set_index('time', inplace=True)
df.head()

## Strategy and Signal Generation

Define the SMA crossover strategy and generate buy/sell signals.


In [ ]:
sma_fast = 5
sma_slow = 20

df['sma_fast'] = df['close'].rolling(sma_fast).mean()
df['sma_slow'] = df['close'].rolling(sma_slow).mean()

# Generate signals
df['signal'] = 0
df.loc[df['sma_fast'] > df['sma_slow'], 'signal'] = 1  # Buy signal
df.loc[df['sma_fast'] < df['sma_slow'], 'signal'] = -1  # Sell signal
df['positions'] = df['signal'].diff()

# Plot to visualize
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['close'], label='Close')
plt.plot(df.index, df['sma_fast'], label=f'SMA {sma_fast}')
plt.plot(df.index, df['sma_slow'], label=f'SMA {sma_slow}')
plt.scatter(df.index, df['close'], c=df['positions'], cmap='viridis')
plt.legend()
plt.show()

## Trade Execution

Automatically execute trades based on the generated signals.


In [ ]:
def execute_trade(position, volume=1.0):
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f'Failed to get {symbol} info')
        return

    if not symbol_info.visible:
        if not mt5.symbol_select(symbol, True):
            print(f'Failed to select symbol {symbol}')
            return

    request = {
        'action': mt5.TRADE_ACTION_DEAL,
        'symbol': symbol,
        'volume': volume,
        'type': mt5.ORDER_TYPE_BUY if position == 1 else mt5.ORDER_TYPE_SELL,
        'price': mt5.symbol_info_tick(symbol).ask if position == 1 else mt5.symbol_info_tick(symbol).bid,
        'sl': 10 * mt5.symbol_info(symbol).point,
        'tp': 20 * mt5.symbol_info(symbol).point,
        'deviation': 2,
        'magic': 123456,
        'comment': 'SMA Crossover',
        'type_time': mt5.ORDER_TIME_GTC,
        'type_filling': mt5.ORDER_FILLING_AON,
    }
    result = mt5.order_send(request)
    
    if result.retcode != mt5.TRADE_RETCODE_DONE:
        print(f'Failed to execute order: {result.retcode}')
    else:
        print(f'Order executed successfully for {symbol}')

# Execute trades
for index, row in df[df['positions'] != 0].iterrows():
    execute_trade(row['positions'])

## Running the Bot

Include a loop to run the strategy at specified intervals (commented out to prevent accidental execution).


In [ ]:
# while True:
#     now = datetime.now()
#     if now.isoweekday() in range(1, 6) and now.hour in range(1, 23):
#         # Fetch data, calculate signals, execute trades
#         pass
#     time.sleep(60)


## Conclusion

This notebook has demonstrated how to create a high-risk leveraged trading bot using a simple moving average crossover strategy in MetaTrader 5.
